In [1]:
%matplotlib inline
from detail import Detail
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
import pylab
import json
import random
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
# print python version:
import sys 
print(sys.version)

2.7.13 |Anaconda custom (x86_64)| (default, Dec 20 2016, 23:05:08) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]


In [16]:
# annFile='../../json/PASCAL_trainval_merged.json' # annotations - out of date
annFile='../../json/trainval_withkeypoints.json'
imgDir='../../VOCdevkit/VOC2010/JPEGImages' # jpeg images
phase='trainval'

In [17]:
# initialize detail api for instance annotations
detailsGt = Detail(annFile, imgDir, phase)

print('\n***info***')
detailsGt.info()

loading annotations into memory...
JSON root keys:[u'info', u'annos_occlusion', u'annos_boundary', u'annos_joints', u'annos_segmentation', u'parts', u'images', u'categories']
Done (t=12.72s)
creating index...
index created! (t=5.68s)

***info***
contributor: https://sites.google.com/view/pasd
date_created: 2017-07-18
version: 3.0
description: This is the 3.0 version of the CVPR 2017 PASCAL in Detail dataset.
year: 2017


## Load eval module for keypoints

In [18]:
from detail import detaileval_kpt # work with python2, fail with python3
detaileval = detaileval_kpt.DetailEvalKpt(detailsGt)

Total category #1, total img #3547 


## Fake a result json for evaluation

In [43]:
gt_imgs_human = detailsGt.getImgs(cats=['person']);
detailRes = []
count = 0
for gt_img in gt_imgs_human:
    '''
    if(2 in gt_img['categories']):
        detailRes[-1]['category_score'][0] = 1
    if(23 in gt_img['categories'] and count < 200):
        count += 1
        detailRes[-1]['category_score'][1] = 1
    '''
    for k in gt_img['keypoints']:
        detailRes.append(
            {'image_id': gt_img['image_id'], 
             'score': 1, 
             'category_id': 284,
             # randomly generate ketpoint in range 0-400 for testing
             'keypoints': np.ndarray.tolist(400*np.random.rand(42))})
        skeleton_id = k
        # load groundtruth to get AP 1
        # detailRes[-1]['keypoints'] = detailsGt.getKptAnno(int(skeleton_id))['keypoints'] # k['keypoints']
    
result = {}
result['pred'] = detailRes
detail_res_file = '../../res/detail_kpt_fake_result.json'
json.dump(result, open(detail_res_file, 'w'), indent = 4)

In [44]:
##### start evaluation
from detail import detaileval_kpt
detaileval = detaileval_kpt.DetailEvalKpt(detailsGt)
detailRes = json.load(open('../../res/detail_kpt_fake_result.json', 'r'))['pred']
detaileval.loadRes(detailRes)

Total category #1, total img #3547 


In [40]:
detaileval.evaluate()

DONE (t=2.49s).


In [41]:
detaileval.accumulate()

Accumulating evaluation results...
DONE (t=0.11s).


In [42]:
detaileval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.471
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.837
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.504
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.017
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.480
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.909
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.694
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.015
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.607
